In [1]:
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd

In [2]:
def drop_by_extension(my_list, extension):
    filtered_list = []
    for item in my_list:
        if not item.endswith(extension.lower()):
            filtered_list.append(item)
    return filtered_list

In [3]:
def get_scan_mask(root, patient):
    dir = root + patient + '/ses-1/anat/'
    dummy = os.listdir(dir)

    scan = dummy[np.argmax(['T1w' in x for x in dummy])]
    mask = dummy[np.argmax(['mask' in x for x in dummy])]

    scan = dir + scan
    mask = dir + mask

    reader = sitk.ImageFileReader()
    reader.SetImageIO("NiftiImageIO")

    reader.SetFileName(scan)
    scan = reader.Execute()

    reader.SetFileName(mask)
    mask = reader.Execute()

    return (scan, mask)

In [4]:
def get_bound_box(input_mask):
    #reader = sitk.ImageFileReader()
    caster = sitk.CastImageFilter()
    lsif = sitk.LabelShapeStatisticsImageFilter()
    
    caster.SetOutputPixelType(sitk.sitkInt8)
    input_mask_int = caster.Execute(input_mask)
    
    lsif.Execute(input_mask_int)

    try:
        boundingBox = np.array(lsif.GetBoundingBox(1))

    except:
        boundingBox = np.array([]) #return nothing
    
    return boundingBox

In [5]:
def relative_center_size(scan, bound_box):
    scan_dim = scan.GetSize()
    #print(scan_dim, bound_box)
    
    x_len_bb = bound_box[3]
    y_len_bb = bound_box[4]
    z_len_bb = bound_box[5]

    x_len_scan = scan_dim[0]
    y_len_scan = scan_dim[1]
    z_len_scan = scan_dim[2]

    x_frac_len = x_len_bb / x_len_scan
    y_frac_len = y_len_bb / y_len_scan
    z_frac_len = z_len_bb / z_len_scan

    x_center_bb = bound_box[0] + x_len_bb/2
    y_center_bb = bound_box[1] + y_len_bb/2
    z_center_bb = bound_box[2] + y_len_bb/2

    x_frac_cen = x_center_bb / x_len_scan
    y_frac_cen = y_center_bb / y_len_scan
    z_frac_cen = z_center_bb / z_len_scan

    #print(x_frac_len, y_frac_len, z_frac_len)
    #print(x_frac_cen, y_frac_cen, z_frac_cen)

    return np.array([z_frac_cen, x_frac_cen, y_frac_cen, z_frac_len, x_frac_len, y_frac_len])


In [6]:
path = '/Volumes/Ex_SSD_2/ATLAS/ATLAS_2/Training/'

center_list = os.listdir(path)
center_list = drop_by_extension(center_list, 'json')

bb_list=[]
columns = ['patient_name', 'scan_dimensons', 'class', 'bb_dimensions', 'relative_bb_size']

for center in center_list:
    patient_list = os.listdir(path + center + '/')
    patient_list = drop_by_extension(patient_list, 'json')
    
    for patient in patient_list:
        scan = get_scan_mask(path + center + '/', patient)[0]
        mask = get_scan_mask(path + center + '/', patient)[1]
        bb = get_bound_box(mask)

        #bb format [xstart, ystart, zstart, xsize, ysize, zsize]

        if bb.size == 0:
            continue

        
        #print(patient, scan.GetSize(), bb, relative_center_size(scan, bb))
        bb_list.append(pd.Series([patient, scan.GetSize(), 1, bb, relative_center_size(scan, bb)], index=columns))

bb_df = pd.DataFrame(bb_list)
bb_df.to_csv('relative_bb_size.csv')